In [ ]:
import random
import math
from sklearn.cluster import DBSCAN
import pandas as pd

def calculate_distance(point1, point2):
    northing_diff = point2[0] - point1[0]
    easting_diff = point2[1] - point1[1]
    return math.sqrt(northing_diff**2 + easting_diff**2)

def calculate_cost(allocation, destinations):
    total_cost = 0
    for source_point in allocation:
        min_distance = float('inf') 
        for destination_point in destinations:
            distance = calculate_distance(source_point, destination_point)
            if distance < min_distance:
                min_distance = distance
        total_cost += random.randint(1, 100) 
    return total_cost

def heuristic_random_generation(n, m, num_trials, tolerance, destinations):
    mean_cost = 100
    std_dev_cost = 20
    total_allocations = math.comb(n, m)
    best_allocations = []
    for _ in range(num_trials):
        allocation = random.sample(destinations, m)
        cost = calculate_cost(allocation, destinations)
        if abs(cost - mean_cost) <= tolerance * std_dev_cost:
            best_allocations.append((allocation, cost))
    return best_allocations

df = pd.read_excel('updateNandE.xlsx',header=0)
n = 117  # Number of possible destination points
m = 10   # Number of source locations (make sure m <= n)
num_trials = 1000
tolerance = 10
# You can adjust this tolerance based on your problem's requirements.

# Assuming you have a list of all destination points with northing and easting coordinates.
destinations = list(zip(df.Northing, df.Easting))
assert m <= len(destinations), "The number of source locations (m) should be less than or equal to the number of destinations (n)."

# Apply DBSCAN to cluster the destination points
dbscan = DBSCAN(eps=1000, min_samples=2)
cluster_labels = dbscan.fit_predict(destinations)

# Collect points for each cluster
cluster_points = {}
for cluster_label, point in zip(cluster_labels, destinations):
    if cluster_label not in cluster_points:
        cluster_points[cluster_label] = []
    cluster_points[cluster_label].append(point)

# Optimize allocation within each cluster
best_allocations = []
for cluster_label, cluster_destinations in cluster_points.items():
    if len(cluster_destinations) >= m:
        cluster_allocations = heuristic_random_generation(len(cluster_destinations), m, num_trials, tolerance, cluster_destinations)
        best_allocations.extend(cluster_allocations)

# Sort the best_allocations based on the cost in ascending order.
best_allocations.sort(key=lambda x: x[1])

# Print the top solutions with the minimum cost.
for i, (allocation, cost) in enumerate(best_allocations[:1]):
    print(f"Solution {i + 1}: Allocation = {allocation}, Cost = {cost}")